# Performance Evaluation Using S&P 500 Data

In this section, we will extend our understanding of performance metrics by applying them to real S&P 500 data. This will include calculating Sharpe, Treynor, Sortino ratios, Alpha, Beta, Information Ratio, Maximum Drawdown, and Tracking Error. We will use different time windows and lengths to analyze how these metrics evolve over time.


In [1]:
%pip install -q yfinance pandas numpy matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

# Fetch historical data for S&P 500 and Nasdaq-100 from Yahoo Finance
sp500 = yf.download('^GSPC', start='2020-01-01', end='2023-01-01')['Adj Close']
nasdaq100 = yf.download('^NDX', start='2020-01-01', end='2023-01-01')['Adj Close']

# Calculate daily returns
sp500_returns = sp500.pct_change().dropna()
nasdaq100_returns = nasdaq100.pct_change().dropna()

# Set the risk-free rate (e.g., 0.01 for 1%)
risk_free_rate = 0.01 / 252  # Adjust for daily returns

# Display the first few rows of the returns for the S&P 500
print("Returns for the S&P 500:")
print(sp500_returns.head())

# Display the first few rows of the returns for the Nasdaq-100
print("\nReturns for the Nasdaq-100:")
print(nasdaq100_returns.head())


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Returns for the S&P 500:
Date
2020-01-03   -0.007060
2020-01-06    0.003533
2020-01-07   -0.002803
2020-01-08    0.004902
2020-01-09    0.006655
Name: Adj Close, dtype: float64

Returns for the Nasdaq-100:
Date
2020-01-03   -0.008827
2020-01-06    0.006211
2020-01-07   -0.000234
2020-01-08    0.007452
2020-01-09    0.008669
Name: Adj Close, dtype: float64


## Calculate Performance Metrics

We will calculate the following metrics for the S&P 500 over different time periods:
- Sharpe Ratio
- Treynor Ratio
- Sortino Ratio
- Alpha and Beta
- Information Ratio
- Maximum Drawdown
- Tracking Error


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf

# Fetch historical data for S&P 500
sp500 = yf.download('^GSPC', start='2018-01-01', end='2023-01-01')['Adj Close']

# Calculate daily returns
returns = sp500.pct_change().dropna()

# Define risk-free rate
risk_free_rate = 0.01 / 252  # Adjust for daily returns

# Function to calculate Sharpe Ratio
def calculate_sharpe_ratio(returns, risk_free_rate):
    return (returns.mean() - risk_free_rate) / returns.std()

# Function to calculate Treynor Ratio (assuming beta = 1 for simplicity)
def calculate_treynor_ratio(returns, risk_free_rate, beta=1):
    return (returns.mean() - risk_free_rate) / beta

# Function to calculate Sortino Ratio (using downside deviation)
def calculate_sortino_ratio(returns, risk_free_rate):
    downside_returns = returns[returns < 0]
    downside_std_dev = downside_returns.std()
    return (returns.mean() - risk_free_rate) / downside_std_dev

# Calculate and display the metrics for different time windows
for period in ['1y', '3y', '5y']:
    period_returns = returns[-int(period[:-1])*252:]  # Approximate trading days per year
    sharpe_ratio = calculate_sharpe_ratio(period_returns, risk_free_rate)
    treynor_ratio = calculate_treynor_ratio(period_returns, risk_free_rate)
    sortino_ratio = calculate_sortino_ratio(period_returns, risk_free_rate)

    print(f"Performance Metrics for the last {period}:")
    print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
    print(f"Treynor Ratio: {treynor_ratio:.4f}")
    print(f"Sortino Ratio: {sortino_ratio:.4f}")
    print("-" * 50)



[*********************100%%**********************]  1 of 1 completed

Performance Metrics for the last 1y:
Sharpe Ratio: -0.0522
Treynor Ratio: -0.0008
Sortino Ratio: -0.0832
--------------------------------------------------
Performance Metrics for the last 3y:
Sharpe Ratio: 0.0198
Treynor Ratio: 0.0003
Sortino Ratio: 0.0246
--------------------------------------------------
Performance Metrics for the last 5y:
Sharpe Ratio: 0.0244
Treynor Ratio: 0.0003
Sortino Ratio: 0.0296
--------------------------------------------------


## Calculate Alpha, Beta, and Information Ratio

Now, let's move on to calculate Alpha, Beta, and Information Ratio for the S&P 500 against the market itself (assuming the market return is the S&P 500 return as a proxy).


Let's start by defining the Alpha and Beta functions.


In [4]:
# Function to calculate Alpha and Beta
def calculate_alpha_beta(portfolio_returns, market_returns, risk_free_rate):
    covariance = np.cov(portfolio_returns, market_returns)[0][1]
    beta = covariance / market_returns.var()
    alpha = portfolio_returns.mean() - (risk_free_rate + beta * (market_returns.mean() - risk_free_rate))
    return alpha, beta

# Function to calculate Information Ratio
def calculate_information_ratio(portfolio_returns, market_returns):
    tracking_error = np.std(portfolio_returns - market_returns)
    if tracking_error == 0:
        return np.nan  # Avoid division by zero
    return (portfolio_returns.mean() - market_returns.mean()) / tracking_error



In [5]:
# Calculate Alpha and Beta
alpha, beta = calculate_alpha_beta(nasdaq100_returns, sp500_returns, risk_free_rate)

# Calculate Information Ratio
information_ratio = calculate_information_ratio(nasdaq100_returns, sp500_returns)

# Display the results
print(f"Alpha: {alpha:.4f}")
print(f"Beta: {beta:.4f}")
print(f"Information Ratio: {information_ratio:.4f}")


Alpha: 0.0001
Beta: 1.1061
Information Ratio: 0.0159


### Performance Metrics Analysis: Nasdaq-100 vs. S&P 500

In this analysis, we compared the performance of the Nasdaq-100 index against the S&P 500 index using key metrics: Alpha, Beta, and Information Ratio.

- **Alpha**: This metric indicates the excess return of the Nasdaq-100 relative to the S&P 500 after accounting for the risk (Beta). A positive Alpha suggests that the Nasdaq-100 outperformed the S&P 500 on a risk-adjusted basis, while a negative Alpha indicates underperformance.

- **Beta**: This metric measures the sensitivity of the Nasdaq-100 to market movements represented by the S&P 500. A Beta greater than 1 implies that the Nasdaq-100 is more volatile than the S&P 500, while a Beta less than 1 indicates less volatility.

- **Information Ratio**: This metric evaluates the consistency of the excess returns of the Nasdaq-100 relative to the S&P 500. A higher Information Ratio indicates a better risk-adjusted return.

By analyzing these metrics, investors can gain insights into the relative performance and risk characteristics of the Nasdaq-100 compared to the broader market represented by the S&P 500.


## Maximum Drawdown and Tracking Error

Finally, we'll calculate the Maximum Drawdown and Tracking Error for the S&P 500 over the given period.


In [6]:
# Function to calculate Maximum Drawdown
def calculate_max_drawdown(prices):
    peak = prices.expanding(min_periods=1).max()
    drawdown = (prices / peak) - 1
    max_drawdown = drawdown.min()
    return max_drawdown

# Function to calculate Tracking Error
def calculate_tracking_error(portfolio_returns, benchmark_returns):
    return np.std(portfolio_returns - benchmark_returns)

# Use the S&P 500 data you fetched previously
max_drawdown = calculate_max_drawdown(sp500)
tracking_error = calculate_tracking_error(returns, returns)  # Using returns as a proxy for simplicity

print(f"Maximum Drawdown: {max_drawdown:.4f}")
print(f"Tracking Error: {tracking_error:.4f}")



Maximum Drawdown: -0.3392
Tracking Error: 0.0000


# Conclusion

We have successfully calculated various performance metrics for the S&P 500 using real market data. These metrics help us understand the risk and return characteristics of the portfolio over different time periods, providing valuable insights for investment decision-making.
